In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [21]:
# Load the CSV file into a Pandas dataframe
df = pd.read_csv('Train_set.csv')

# Display dimensions of the dataset
print(df.shape)

(87554, 189)


In [22]:
# Display first 5 rows of the dataset
df.head()

,ID,T0,T1,T2,T3,T4,T5,T6,T7,T8,...,T178,T179,T180,T181,T182,T183,T184,T185,T186,Class
0,0,0.965812,0.792023,0.116809,0.000000,0.162393,0.213675,0.264957,0.247863,0.270655,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,1.000000,0.597015,0.000000,0.109453,0.094527,0.084577,0.074627,0.094527,0.114428,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,0.831382,0.714286,0.491803,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,1.000000,0.837705,0.236066,0.037705,0.252459,0.329508,0.319672,0.306557,0.304918,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,0.868590,0.448718,0.490385,0.477564,0.461538,0.455128,0.416667,0.304487,0.182692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [23]:
# Display all values in Class column
df['Class'].unique()

array([0, 4, 1, 2, 3])

In [9]:
# Count the number of missing values in each column
print(df.isnull().sum())

ID          0
T0        474
T1        919
T2       1327
T3       1711
         ... 
T183     4898
T184     4956
T185     4928
T186     4909
Class       0
Length: 189, dtype: int64


In [10]:
# Display the avg of missing values in each column and sort them
print(df.isnull().mean().sort_values(ascending=False))

T76      0.058558
T77      0.058387
T69      0.058318
T75      0.058261
T78      0.058044
           ...   
T2       0.015156
T1       0.010496
T0       0.005414
ID       0.000000
Class    0.000000
Length: 189, dtype: float64


In [11]:
# Display the avg of missing values in each row and sort them
print(df.isnull().mean(axis=1).sort_values(ascending=False))

41960    0.105820
46987    0.105820
18223    0.105820
23724    0.105820
34985    0.105820
           ...   
16218    0.005291
37495    0.005291
22590    0.005291
33657    0.005291
40458    0.005291
Length: 87554, dtype: float64


In [24]:
# Impute NaN values with the median value of each column
df.fillna(df.median(), inplace=True)

In [25]:
# Display dimensions of the dataset
df.head()

,ID,T0,T1,T2,T3,T4,T5,T6,T7,T8,...,T178,T179,T180,T181,T182,T183,T184,T185,T186,Class
0,0,0.965812,0.792023,0.116809,0.000000,0.162393,0.213675,0.264957,0.247863,0.270655,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,1.000000,0.597015,0.000000,0.109453,0.094527,0.084577,0.074627,0.094527,0.114428,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,0.831382,0.714286,0.491803,0.166197,0.148022,0.158824,0.145332,0.144465,0.149909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,1.000000,0.837705,0.236066,0.037705,0.252459,0.329508,0.319672,0.306557,0.304918,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,0.868590,0.448718,0.490385,0.477564,0.461538,0.455128,0.416667,0.304487,0.182692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [14]:
# Separate the ID and class columns from the input features
ids = df['ID']
y = df['Class']
X = df.drop(['ID', 'Class'], axis=1)

# Standardize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape the input features into a 3D array for input to the LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [16]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, (h_n, c_n) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Convert the preprocessed data to PyTorch tensors
X_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(y.values).long()

# Create a PyTorch DataLoader to handle batching and shuffling of the data
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the device to use for training (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize the model and move it to the device
model = LSTMModel(input_size=1, hidden_size=128, num_layers=1, output_size=5).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train the model
num_epochs = 40
for epoch in range(num_epochs):
    for X_batch, y_batch in dataloader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        # Zero out the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    # Print the loss and validation accuracy at the end of each epoch
    with torch.no_grad():
        total_loss = 0
        total_correct = 0
        total_samples = 0
        for X_val, y_val in dataloader:
            X_val = X_val.to(device)
            y_val = y_val.to(device)
            outputs = model(X_val)
            total_loss += criterion(outputs, y_val).item() * X_val.shape[0]
            total_correct += (outputs.argmax(dim=1) == y_val).sum().item()
            total_samples += X_val.shape[0]
        val_loss = total_loss / total_samples
        val_acc = total_correct / total_samples
    print(f"Epoch {epoch+1}/{num_epochs}: train_loss={loss.item():.4f} val_loss={val_loss:.4f} val_acc={val_acc:.4f}")


Epoch 1/40: train_loss=0.0612 val_loss=0.4370 val_acc=0.8752
Epoch 2/40: train_loss=0.0654 val_loss=0.3524 val_acc=0.9021
Epoch 3/40: train_loss=0.0176 val_loss=0.2781 val_acc=0.9290
Epoch 4/40: train_loss=0.4930 val_loss=0.2495 val_acc=0.9316
Epoch 5/40: train_loss=0.0298 val_loss=0.2193 val_acc=0.9412
Epoch 6/40: train_loss=0.0136 val_loss=0.1859 val_acc=0.9480
Epoch 7/40: train_loss=0.0520 val_loss=0.2621 val_acc=0.9253
Epoch 8/40: train_loss=0.0487 val_loss=0.1534 val_acc=0.9577
Epoch 9/40: train_loss=0.0872 val_loss=0.1339 val_acc=0.9631
Epoch 10/40: train_loss=0.0085 val_loss=0.1330 val_acc=0.9645
Epoch 11/40: train_loss=0.0152 val_loss=0.1201 val_acc=0.9678
Epoch 12/40: train_loss=0.2307 val_loss=0.1125 val_acc=0.9697
Epoch 13/40: train_loss=0.0442 val_loss=0.1085 val_acc=0.9709
Epoch 14/40: train_loss=0.0024 val_loss=0.1102 val_acc=0.9703
Epoch 15/40: train_loss=0.0032 val_loss=0.1018 val_acc=0.9720
Epoch 16/40: train_loss=0.0317 val_loss=0.0981 val_acc=0.9729
Epoch 17/40: trai

In [27]:
# Define the path where you want to save the model
model_path = 'my_lstm_model.pth'

# Save the model
#torch.save(model.state_dict(), model_path)

In [28]:
# Load the saved model
loaded_model = LSTMModel(1, 128, 1, 5)
loaded_model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [52]:
# Load test data into a pandas DataFrame
test_data = pd.read_csv("Test_set.csv")

In [53]:
# Impute NaN values with the median value of each column
test_data.fillna(test_data.median(), inplace=True)

In [54]:
test_data.head()

,ID,T0,T1,T2,T3,T4,T5,T6,T7,T8,...,T177,T178,T179,T180,T181,T182,T183,T184,T185,T186
0,0,0.966581,0.802057,0.077121,0.000000,0.161954,0.244216,0.254499,0.262211,0.264782,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.000000,0.644444,0.031111,0.026667,0.151111,0.128889,0.080000,0.080000,0.084444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1.000000,0.965318,0.734104,0.462428,0.271676,0.156069,0.150289,0.173410,0.173410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1.000000,0.998792,0.960145,0.896135,0.898551,0.873188,0.766908,0.591787,0.327295,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.009836,0.000000,0.163934,0.242623,0.327869,0.350820,0.331148,0.331148,0.354098,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
# Drop irrelevant columns
test_data = test_data.drop(columns=["ID"])

In [56]:
# Standardize the input features
scaler = StandardScaler()
X = scaler.fit_transform(test_data)

# Reshape the input features into a 3D array for input to the LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Convert the numpy array to a PyTorch tensor
test_tensor = torch.Tensor(X)

# Create a DataLoader object for the test data
test_loader = torch.utils.data.DataLoader(test_tensor, batch_size=32, shuffle=False)


In [58]:
# Set the device to be used for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the same device as the input tensor
loaded_model = loaded_model.to(device)

In [61]:
# Set the model to evaluation mode
loaded_model.eval()

# Create an empty list to store the predicted classes
pred_classes = []

# Loop through the test data in batches using the test loader
for X_batch in test_loader:
    # Move the batch to the device
    X_batch = X_batch.to(device)

    # Compute the model's predicted class for the batch
    outputs = loaded_model(X_batch)
    _, predicted = torch.max(outputs.data, 1)

    # Convert the predicted class tensor to a numpy array
    predicted = predicted.cpu().numpy()

    # Append the predicted classes to the list
    pred_classes.append(predicted)

# Convert the list of predicted classes to a numpy array
pred_classes = np.concatenate(pred_classes)

# Create a new pandas DataFrame with the predicted classes and an ID column (if necessary)
ids = np.arange(len(pred_classes))
df = pd.DataFrame({'ID': ids, 'Pred_Class': pred_classes})

# Save the DataFrame as a CSV file
df.to_csv('sample_submission1.csv', index=False)



In [60]:
result = pd.read_csv("sample_submission.csv")

result.head()

,ID,Pred_Class
0,1,0
1,2,0
2,3,0
3,4,2
4,5,0
